<a href="https://colab.research.google.com/github/arnaidu/RNN-Attention-COVID19/blob/main/CSC413_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC413 Project Code

# Utils

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import abc 
from itertools import combinations, permutations
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from tqdm import tqdm
import os

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

for i in range(1, 5): ## make directories for experiment
  directory = 'experiment{}'.format(i)
  try:
    os.stat(directory)
  except:
      os.mkdir(directory)

# Import Data

In [ ]:
## Import csv file
url = "https://data.ontario.ca/dataset/f4f86e54-872d-43f8-8a86-3892fd3cb5e6/resource/ed270bb8-340b-41f9-a7c6-e8ef587e6d11/download/covidtesting.csv"
df = pd.read_csv(url)
## Delete the last 3 ('Total_Lineage...') columns
df = df.drop(columns=['Total_Lineage_B.1.1.7', 'Total_Lineage_B.1.351', 'Total_Lineage_P.1'])
## Also delete the 'legacy variables' (ie, Confirmed Negative, Presumptive Positive, Presumptive Negative) -- these were only reported at the very beginning
df = df.drop(columns=['Confirmed Negative', 'Presumptive Positive', 'Presumptive Negative'])
## Also delete the "new" stats (only started reporting from March 30, 2021)
df = df.drop(columns=['Number of patients in ICU, testing positive for COVID-19', 'Number of patients in ICU, testing negative for COVID-19', 'Num. of patients in ICU on a ventilator testing positive', 'Num. of patients in ICU on a ventilator testing negative'])

print(df.columns)
['Total Cases', 'Deaths'] ## cumulative
['Positive', 'New Deaths'] ## not cumulative

BASIC_FEATURES = ['Positive', 'Total Cases', 'Resolved', 'Deaths']
TEST_FEATURES = ['AFT','TTC', 'PPT', 'UI']
HOSPITAL_FEATURES = ['Hospital', 'ICU', 'Ventilator']
LTC_FEATURES = ['TPLTCR', 'TPLTCHCW', 'TLCTRD', 'TLTCHCWD']
NEW_LTC_FEATURES = ['PLTCR', 'PLTCHCW', 'LCTRD', 'LTCHCWD']

## Rename comlumns for simplicity
'''
AFT = Total patients approved for testing as of Reporting Date;
TTC = Total tests completed in the last day;
PPT = Percent positive tests in last day; UI = Under Investigation;
Hospital = Number of patients hospitalized with COVID-19;
ICU = Number of patients in ICU due to COVID-19;
Ventilator = Number of patients in ICU on a ventilator due to COVID-19;
TPLTCR = Total Positive LTC Resident Cases;
TPLTCHCW = Total Positive LTC HCW Cases;
TLCTRD = Total LTC Resident Deaths;
TLTCHCWD = Total LTC HCW Deaths
'''
df = df.rename(columns={'Confirmed Positive': 'Positive',
'Total patients approved for testing as of Reporting Date': 'AFT',
'Total tests completed in the last day': 'TTC',
'Percent positive tests in last day': 'PPT', 'Under Investigation': 'UI',
'Number of patients hospitalized with COVID-19': 'Hospital',
'Number of patients in ICU due to COVID-19': 'ICU',
'Number of patients in ICU on a ventilator due to COVID-19': 'Ventilator',
'Total Positive LTC Resident Cases': 'TPLTCR',
'Total Positive LTC HCW Cases': 'TPLTCHCW',
'Total LTC Resident Deaths': 'TLCTRD', 'Total LTC HCW Deaths': 'TLTCHCWD'})

## Delete data before May 19, 2020 (day that all stats have at least one entry)
df = df.drop(df[df['Reported Date'] < '2020-05-19'].index)
df = df.reset_index(drop=True)
df_date = df['Reported Date']
df = df.drop(columns=['Reported Date'])

# Interpolate middle NaNs with average of closest known values
df = df.interpolate(method='linear', axis=0, limit_direction='both')

## Change cumulative counts to differences for LTC features
df[NEW_LTC_FEATURES] = df[LTC_FEATURES].sub(df[LTC_FEATURES].shift())

## Change cumulative death counts to differences
df['New Deaths'] = df['Deaths'].sub(df['Deaths'].shift())

## Replace negative values with 0 -- improper cumulative counts
df[df < 0] = 0
df['Reported Date'] = df_date
df = df[1:]
print(df.columns)

## Reorder columns
# df = df[['Positive', 'Total Cases', 'Resolved', 'Deaths', 'New Deaths'] + TEST_FEATURES + HOSPITAL_FEATURES + LTC_FEATURES + NEW_LTC_FEATURES + ['Reported Date']]

df = df[['Positive', 'Total Cases', 'Resolved', 'Deaths'] + TEST_FEATURES + HOSPITAL_FEATURES + LTC_FEATURES + ['Reported Date']]


ALL_DATES = df['Reported Date'].values

Index(['Reported Date', 'Confirmed Positive', 'Resolved', 'Deaths',
       'Total Cases',
       'Total patients approved for testing as of Reporting Date',
       'Total tests completed in the last day',
       'Percent positive tests in last day', 'Under Investigation',
       'Number of patients hospitalized with COVID-19',
       'Number of patients in ICU due to COVID-19',
       'Number of patients in ICU on a ventilator due to COVID-19',
       'Total Positive LTC Resident Cases', 'Total Positive LTC HCW Cases',
       'Total LTC Resident Deaths', 'Total LTC HCW Deaths'],
      dtype='object')
Index(['Positive', 'Resolved', 'Deaths', 'Total Cases', 'AFT', 'TTC', 'PPT',
       'UI', 'Hospital', 'ICU', 'Ventilator', 'TPLTCR', 'TPLTCHCW', 'TLCTRD',
       'TLTCHCWD', 'PLTCR', 'PLTCHCW', 'LCTRD', 'LTCHCWD', 'New Deaths',
       'Reported Date'],
      dtype='object')


# Preprocess Data

## Cluster Data

In [ ]:
def cluster_data(df, k, t, in_features=None, out_features=['Positive', 'Resolved', 'New Deaths'],
                 separate=True, moving_average=False, m=3):
  """Cluster data for t days ahead prediction with condition window k
  ie, target is t days ahead of the last day in the condition window
  
  df: Dataframe of cleaned data
  k: Condition window size
  t: Number of days ahead for prediction
  in_features: List of input features to be in X
  out_features: List of output features to be in y
  moving_average: Whether or not to model the moving average (form of data smoothing)
      - Then, the condition window will be k consecutive moving averages and we predict 
      the t day's ahead moving average
  m: If moving_average is True, the number of days involved in the moving average

  return: 
  - X (# time steps - t, t, len(in_features)) -- torch.Tensor (note: on cpu; must put on gpu in a later step)
     - if in_features is None, in_features = all features
  - y (# time steps - t, len(out_features)) -- torch.Tensor
  """
  df_date = df['Reported Date']
  df = df.drop(columns='Reported Date')

  ## Note slicing in pandas (in this case), is inclusive of the end index
  if separate:
      if moving_average:
          if in_features is None:
              in_features = list(set(df.columns) - set(out_features))
          clustered_in = torch.as_tensor([df.loc[s:s+m-1, in_features].values for s in range(1, len(df) - m + 2)])
          clustered_in = torch.mean(clustered_in, dim=1).squeeze()
          clustered_out = torch.as_tensor([df.loc[s:s+m-1, out_features].values for s in range(1, len(df) - m + 2)])
          clustered_out = torch.mean(clustered_out, dim=1).squeeze()
          X = torch.stack([clustered_in[s:s+k] for s in range(len(clustered_in) - t - k + 1)], dim=0)
          X_tilde = torch.stack([clustered_out[s:s+k] for s in range(len(clustered_out) - t - k + 1)], dim=0)
          y = clustered_out[k + t - 1:]
          y_cumul = torch.stack([clustered_out[d:d+t] for d in range(k, len(clustered_out) - t + 1)], dim=0).sum(dim=1)
          dates = df_date.loc[(m // 2) + k + t:].values[:-m//2 + 1]
      else:
          if in_features is None:
              in_features = list(set(df.columns) - set(out_features))
          X = torch.as_tensor([df.loc[s:s+k-1, in_features].values for s in range(1, len(df) - t - k + 2)])
          X_tilde = torch.as_tensor([df.loc[s:s+k-1, out_features].values for s in range(1, len(df) - t - k + 2)])
          y = torch.as_tensor(df[out_features].loc[k + t:].values)
          y_cumul = torch.stack([torch.as_tensor(df[out_features].loc[d:d+t-1].values) for d in range(k+1, len(df) - t + 2)], dim=0).sum(dim=1)
          dates = df_date.loc[k + t:].values
  else:
      if moving_average:
          if in_features is not None:
              clustered_in = torch.as_tensor([df.loc[s:s+m-1, in_features].values for s in range(1, len(df) - m + 2)])
          else:
              clustered_in = torch.as_tensor([df.loc[s:s+m-1].values for s in range(1, len(df) - m + 2)])
          clustered_in = torch.mean(clustered_in, dim=1).squeeze()
          clustered_out = torch.as_tensor([df.loc[s:s+m-1, out_features].values for s in range(1, len(df) - m + 2)])
          clustered_out = torch.mean(clustered_out, dim=1).squeeze()
          X = torch.stack([clustered_in[s:s+k] for s in range(len(clustered_in) - t - k + 1)], dim=0)
          y = clustered_out[k + t - 1:]
          y_cumul = torch.stack([clustered_out[d:d+t] for d in range(k, len(clustered_out) - t + 1)], dim=0).sum(dim=1)
          dates = df_date.loc[(m // 2) + k + t:].values[:-m//2 + 1]
      else:
          if in_features is None:
              in_features = df.columns()
          X = torch.as_tensor([df.loc[s:s+k-1, in_features].values for s in range(1, len(df) - t - k + 2)])
          y = torch.as_tensor(df[out_features].loc[k + t:].values)
          y_cumul = torch.stack([torch.as_tensor(df[out_features].loc[d:d+t-1].values) for d in range(k+1, len(df) - t + 2)], dim=0).sum(dim=1)
          dates = df_date.loc[k + t:].values
      X_tilde = None

  if len(X.shape) == 2: ## encoder input only 1 feature
      X = X.unsqueeze(-1)
  if X_tilde is not None and len(X_tilde.shape) == 2: ## decoder input only 1 feature
      X_tilde = X_tilde.unsqueeze(-1)
  if len(y.shape) == 1: ## output only 1 feature
      y = y.unsqueeze(-1)
      y_cumul = y_cumul.unsqueeze(-1)

  return X, X_tilde, y, y_cumul, dates, in_features

# You can uncomment this to see how the data looks like
# X, X_tilde, y, y_cumul, dates = cluster_data(df, in_features=['New Deaths'], out_features=['New Deaths'], k=5, t=7, moving_average=True, m=3)
# print(X[:10])
# print(y_cumul)

## Dataset and DataLoader

In [ ]:
class CovidDataset(Dataset):
  def __init__(self, X, X_tilde, y):
    self.X = X
    self.X_tilde = X_tilde
    self.y = y
  
  def __len__(self):
    return self.X.shape[0]
  
  def __getitem__(self, i):
    if self.X_tilde is None:
      return self.X[i], self.y[i]
    else:
      return self.X[i], self.X_tilde[i], self.y[i]

class CovidDataLoader(DataLoader):
  def __init__(self, X, X_tilde, y, batch_size):
    dataset = CovidDataset(X, X_tilde, y)
    super().__init__(dataset, batch_size=batch_size, shuffle=True)

## Train, Validation, Test Split

In [ ]:
def train_val_test_split(X, X_tilde, y, dates):
    """Splits dataset into training, validation and testing"""

    X, y = X.float(), y.float()
    jan_1_idx = np.where(dates == '2021-01-01')[0][0]
    march_1_idx = np.where(dates == '2021-03-01')[0][0]
    april_17_idx = np.where(dates == '2021-04-17')[0][0]

    X_train = X[:jan_1_idx]
    y_train = y[:jan_1_idx]
    X_val = X[jan_1_idx:march_1_idx]
    y_val = y[jan_1_idx:march_1_idx]
    X_test = X[march_1_idx:april_17_idx]
    y_test = y[march_1_idx:april_17_idx]

    if X_tilde is not None:
        X_tilde = X_tilde.float()
        X_tilde_train = X_tilde[:jan_1_idx]
        X_tilde_val = X_tilde[jan_1_idx:march_1_idx]
        X_tilde_test = X_tilde[march_1_idx:april_17_idx]
    else:
        X_tilde_train, X_tilde_val, X_tilde_test = None, None, None

    return X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test

# Different types of RNN architectures

## MyRNN

In [ ]:
class MyRNN(nn.Module):
    def __init__(self, rnn_class, input_size, hidden_size, output_size, num_layers, seq_length, dropout, bidirectional=False, cell_type="VanillaRNN", attn_type=None, temporal_attn=True):
    
        super().__init__()
        self.rnn = rnn_class(input_size, hidden_size, num_layers, seq_length, dropout, bidirectional, cell_type, attn_type)
        self.attn_type = attn_type
        self.forward_attn_weights = None
        self.backward_attn_weights = None
        self.temporal_attn = temporal_attn
        self.temporal_attn_weights = None
        self.bidirectional = bidirectional
        self.hidden_size = hidden_size

        if bidirectional:
            self.temporal_attn_layer = nn.Sequential(nn.Linear(hidden_size * 2, hidden_size), nn.Tanh(), nn.Linear(hidden_size, 1), nn.Softmax(dim=1))
            self.fc = nn.Linear(hidden_size*2, output_size) 
        else:
            self.temporal_attn_layer = nn.Sequential(nn.Linear(hidden_size, hidden_size // 2), nn.Tanh(), nn.Linear(hidden_size // 2, 1), nn.Softmax(dim=1))
            self.fc = nn.Linear(hidden_size, output_size)


    def get_attention_weights(self):
        return (self.forward_attn_weights, self.backward_attn_weights), self.temporal_attn_weights

    def forward(self, X, X_tilde):
        if self.attn_type is None:
            output = self.rnn(X)
        else:
            output, self.forward_attn_weights, self.backward_atnn_weights = self.rnn(X)
        
        if self.temporal_attn:
            self.temporal_attn_weights = self.temporal_attn_layer(output) ## (B, S, 1)
            output = (self.temporal_attn_weights.transpose(2,1) @ output).squeeze() ## (B, 1, S) * (B, S, H) = (B, 1, H)
        else:
            if self.bidirectional:
                h_forward = output[:, -1, :self.hidden_size]
                h_backward = output[:, 0, self.hidden_size:]
                output = torch.cat([h_forward, h_backward], dim=1)
            else:
                output = output[:, -1, :, :]

        output = self.fc(output)
        ## Unnormalize output (we assume that the output features are the first ones in the input)
        output = X_tilde.mean(dim=1) + output * X_tilde.std(dim=1)

        return output


## RNN Without Input Attention

In [ ]:
class RNNWithoutAttention(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, seq_len, dropout, bidirectional=False, cell_type="VanillaRNN", attn_type=None):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.bidirectional = bidirectional

    if cell_type == "VanillaRNN":
      self.rnn = nn.RNN(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

    elif cell_type == "LSTM":
      self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

    else:
      self.rnn = nn.GRU(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

  def forward(self, X):
    """
    Inputs:
      data = the initial data of size (batch_size, seq_length, num_input_features)
    Output:
      output = all hidden states
    """
    ## Note: initial hidden/cell states default to 0

    ## Normalize input data
    mean = X.mean(dim=1).unsqueeze(1)
    std = X.std(dim=1).unsqueeze(1)
    data = (X - mean) / (std + 1e-7)

    output = self.rnn(data)[0]
    
    return output

## RNN with Input Attention

In [ ]:
class RNNWithAttention(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, seq_len, dropout, bidirectional=False, cell_type="VanillaRNN", attn_type="Additive"):
    
    super().__init__()

    self.input_size = input_size
    self.seq_len = seq_len
    self.num_hidden_layers = num_layers
    self.hidden_state_size = hidden_size
    self.dropout = dropout
    self.cell_type = cell_type
    self.attn_type = attn_type
    self.bidirectional = bidirectional
    self.init_submodules()


  def init_submodules(self):
    """Initialize the submodules for encoder with attention"""
    
    if self.cell_type == "VanillaRNN":
      net = [nn.RNNCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.RNNCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    elif self.cell_type == "LSTM":
      net = [nn.LSTMCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.LSTMCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    
    else:
      net = [nn.GRUCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.GRUCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    
    self.rnn_forward = nn.Sequential(*net)

    self.project_to_input_dim_forward = nn.Linear(self.hidden_state_size, self.input_size)
    self.project_to_input_dim_backward = nn.Linear(self.hidden_state_size, self.input_size)

    if self.attn_type == "Additive":
      attn = [AdditiveAttention(hidden_size=self.seq_len, encoder=True) for _ in range(self.num_hidden_layers)]
    elif self.attn_type == "Scaled-Dot":
      attn = [ScaledDotAttention(hidden_size=self.seq_len, hidden_size2=self.input_size, encoder=True)]
      attn.extend([ScaledDotAttention(hidden_size=self.seq_len, hidden_size2 = self.hidden_state_size, encoder=True) for _ in range(self.num_hidden_layers - 1)])
    elif self.attn_type == "Linear":
      attn = [LinearAttention(hidden_size=self.seq_len, encoder=True) for _ in range(self.num_hidden_layers)]
    else:
      attn = [DaRnnAttention(hidden_size=self.seq_len) for _ in range(self.num_hidden_layers)]

    self.attn_forward = nn.Sequential(*attn)

    if self.bidirectional:
      self.rnn_backward = nn.Sequential(*net.copy())
      self.attn_backward = nn.Sequential(*attn.copy()) 
    
    self.forward_attn_weights = None
    self.backward_attn_weights = None

  def forward(self, X):
    """Forward pass of data in RNN with input attention"""
    # Normalize input data
    mean = X.mean(dim=1).unsqueeze(1)
    std = X.std(dim=1).unsqueeze(1)
    data = (X - mean) / (std + 1e-7)

    forward_output, self.forward_attn_weights = self.forward_pass(data)
    if self.bidirectional: ## Get last hidden state
      backward_output, self.backward_attn_weights = self.backward_pass(data)
      output = torch.cat([forward_output, backward_output], dim=2)
    else:
      output = forward_output
    
    return output, self.forward_attn_weights, self.backward_attn_weights

  def forward_pass(self, data, direction='Forward'):
    """Forward pass of rnn with attention in the forward direction"""
    
    attentions_all_layers = []

    if direction == 'Forward':
        attn_net = self.attn_forward
        rnn = self.rnn_forward
        project_to_input_dim = self.project_to_input_dim_forward
    else:
        attn_net = self.attn_backward
        rnn = self.rnn_backward
        project_to_input_dim = self.project_to_input_dim_backward

    ## Iterate over all layers
    for i in range(self.num_hidden_layers):
      attentions = []
      
      ## Get first hidden state
      h_prev = self.get_first_hidden_state(data) ## (batch_size, input_size) or (batch_size, hidden_size) 

      ## Loop over the sequence
      hiddens = []
      for j in range(self.seq_len):
        current_input = data[:, j, :] ## Get input at current time step (batch_size, input_size) or (batch_size, hidden_size)
        data = data.transpose(1, 2) ## (batch_size, input_size, seq_len) or (batch_size, hidden_size, seq_len)

        if self.cell_type == "LSTM":
            if h_prev[0].shape[1] != data.shape[1]:
                h_prev_pr = project_to_input_dim(h_prev[0]) ## (batch_size, input_size)
            else:
                h_prev_pr = h_prev[0]
        else:
            if h_prev.shape[1] != data.shape[1]:
                h_prev_pr = project_to_input_dim(h_prev)
            else:
                h_prev_pr = h_prev

        h_repeated = h_prev_pr.unsqueeze(-1).expand_as(data)  ## (batch_size, input_size, seq_len)

        ## Pass through attention layer
        _, attention_weights = attn_net[i](h_repeated, data, data) ## (batch_size, input_size, 1) or (batch_size, hidden_size, 1)
        ## Pass through rnn
        weighted_input = attention_weights.squeeze(2) * current_input
        h_prev = rnn[i](weighted_input, h_prev) ## (batch_size, hidden_size)
        
        ## Keep important stuff in arrays
        attentions.append(attention_weights) ## (batch_size, hidden_size) 
        if self.cell_type == "LSTM":
          hiddens.append(h_prev[0]) ## (batch_size, hidden_size) 
        else:
          hiddens.append(h_prev)

        ## Revert back to previous orientation
        data = data.transpose(1, 2) ## (batch_size, seq_len, input_size) or (batch_size, seq_len, hidden_size)
      
      ## outputs of previous layer are inputs to the next layer
      data = torch.stack(hiddens, dim=1) ## (batch_size, seq_len, hidden_size)
      attentions = torch.cat(attentions, dim=2) ## (batch_size, hidden_size, seq_len)
      attentions_all_layers.append(attentions)
    
    final_hiddens = data
    ## attentions_all_layers is a list of num_layers tensors of size (batch_size, input or hidden_size, seq_len)
    return final_hiddens, attentions_all_layers
  
  def backward_pass(self, data):
      """RNN in the reverse direction"""
      ## Reverse the input data along the sequence dimension
      data = data.flip(dims=(1,))

      return self.forward_pass(data, direction='Backward')

  def get_first_hidden_state(self, encoder_input):
    ## Just return tensor of zeros or tuple if LSTM
    ## Return shape: (batch_size, hidden_size)
    htilde_tm1 = torch.zeros((encoder_input.shape[0], self.hidden_state_size), device=encoder_input.device)
    if self.cell_type == "LSTM":
      return htilde_tm1, torch.zeros_like(htilde_tm1)
    else:
      return htilde_tm1

# Encoder-Decoder Architectures

## EncoderDecoder

In [ ]:
class EncoderDecoder(nn.Module):
  def __init__(self, encoder_class, decoder_class, input_size, seq_len, encoder_num_hidden_layers, 
               hidden_state_size, output_size, dropout, cell_type,
               enc_attn_type, decoder_num_hidden_layers, dec_attn_type, bidirectional, temporal_attn=True):
    super().__init__()
    self.encoder = encoder_class(input_size, seq_len, encoder_num_hidden_layers, 
               hidden_state_size, dropout=dropout, cell_type=cell_type, attn_type=enc_attn_type)
    
    ## Bidirectional
    if bidirectional:
        self.decoder = decoder_class(output_size, seq_len, decoder_num_hidden_layers, hidden_state_size * 2, cell_type=cell_type, attn_type=dec_attn_type)
        self.temporal_attn_layer = nn.Sequential(nn.Linear(hidden_state_size * 2, hidden_state_size), nn.Tanh(), nn.Linear(hidden_state_size, 1), nn.Softmax(dim=1))
        self.fc = nn.Linear(hidden_state_size * 2, output_size)
    else:
        self.decoder = decoder_class(output_size, seq_len, decoder_num_hidden_layers, hidden_state_size, cell_type=cell_type, attn_type=dec_attn_type)
        self.temporal_attn_layer = nn.Sequential(nn.Linear(hidden_state_size, hidden_state_size // 2), nn.Tanh(), nn.Linear(hidden_state_size // 2, 1), nn.Softmax(dim=1))
        self.fc = nn.Linear(hidden_state_size, output_size)

    self.enc_attn_type = enc_attn_type
    self.dec_attn_type = dec_attn_type

    self.input_attn_weights = None
    self.temporal_attn_weights = None

    self.temporal_attn = temporal_attn

  def get_attention_weights(self):
      return self.input_attn_weights, self.temporal_attn_weights

  def forward(self, encoder_input, decoder_input):
    ## Normalize encoder_input
    enc_mean = encoder_input.mean(dim=1).unsqueeze(1)
    enc_std = encoder_input.std(dim=1).unsqueeze(1)
    encoder_input = (encoder_input - enc_mean) / (enc_std + 1e-7)

    ## Put through encoder
    encoder_output = self.encoder(encoder_input) ## (batch_size, seq_len, hidden_state_size * 2) if no attention
    if self.enc_attn_type is not None: ## (batch_size, seq_len, hidden_state_size) with attention
        encoder_output, self.input_attn_weights = encoder_output

    ## Normalize decoder_input
    dec_mean = decoder_input.mean(dim=1).unsqueeze(1)
    dec_std = decoder_input.std(dim=1).unsqueeze(1)
    decoder_input = (decoder_input - dec_mean) / (dec_std + 1e-7)
    ## Put through decoder
    decoder_output = self.decoder(encoder_output, decoder_input) ## (batch_size, hidden_state_size * 2) if no attention
    if self.dec_attn_type is not None: ## (batch_size, hidden_state_size) if with attention
        decoder_output, final_hidden, _ = decoder_output
    else:
        final_hidden = decoder_output[:, -1, :]

    if self.temporal_attn:
        self.temporal_attn_weights = self.temporal_attn_layer(decoder_output) ## (B, S, 1)
        decoder_output = (self.temporal_attn_weights.transpose(2,1) @ decoder_output).squeeze() ## (B, 1, S) * (B, S, H) = (B, 1, H)
    else:
        decoder_output = final_hidden

    output = self.fc(decoder_output) ## (batch_size, output_size)

    ## "Unnormalize" the decoder output 
    output = dec_mean.squeeze(1) + output * dec_std.squeeze(1)

    return output

## Encoder Without Attention

In [ ]:
class EncoderWithoutAttention(nn.Module):
  def __init__(self, input_size, seq_len, num_hidden_layers, hidden_state_size, dropout=0.1, cell_type='LSTM', attn_type=None):
    super().__init__()

    self.input_size = input_size
    self.seq_len = seq_len
    self.num_hidden_layers = num_hidden_layers
    self.hidden_state_size = hidden_state_size
    self.dropout = dropout
    self.cell_type = cell_type
    self.attn_type = attn_type
    self.init_submodules()

  def init_submodules(self):
    if type == "VanillaRNN":
      self.rnn = nn.RNN(self.input_size, self.hidden_state_size, self.num_hidden_layers, dropout=self.dropout, batch_first=True, bidirectional=True)
    elif type == "LSTM":
      self.rnn = nn.LSTM(self.input_size, self.hidden_state_size, self.num_hidden_layers, dropout=self.dropout, batch_first=True, bidirectional=True)
    else:
      self.rnn = nn.GRU(self.input_size, self.hidden_state_size, self.num_hidden_layers, dropout=self.dropout, batch_first=True, bidirectional=True)
      
  def forward(self, encoder_input):
    return self.rnn(encoder_input)[0] ## returns all hidden states

## Decoder Without Attention

In [ ]:
class DecoderWithoutAttention(nn.Module):
  def __init__(self, output_size, seq_len, num_hidden_layers, hidden_state_size, cell_type='LSTM', attn_type=None):
    super().__init__()

    self.output_size = output_size
    self.seq_len = seq_len
    self.num_hidden_layers = num_hidden_layers
    self.hidden_state_size = hidden_state_size ## Note: this should equal 2 * encoder_hidden_state_size
    self.cell_type = cell_type
    self.attn_type = attn_type
    self.init_submodules()

  def init_submodules(self):
    if self.cell_type == "VanillaRNN":
      self.rnn = nn.RNN(input_size=self.output_size, hidden_size=self.hidden_state_size, 
                        num_layers=self.num_hidden_layers, batch_first=True)

    elif self.cell_type == "LSTM":
      self.rnn = nn.LSTM(input_size=self.output_size, hidden_size=self.hidden_state_size, 
                         num_layers=self.num_hidden_layers, batch_first=True)

    else:
      self.rnn = nn.GRU(input_size=self.output_size, hidden_size=self.hidden_state_size, 
                        num_layers=self.num_hidden_layers, batch_first=True)

  def forward(self, encoder_output, decoder_input):
    htilde_tm1 = self.get_first_hidden_state(encoder_output)
    if self.cell_type == 'LSTM':
        htilde_tm1 = (htilde_tm1, torch.zeros_like(htilde_tm1, device=htilde_tm1.device)) ## initialize cell state with zeroes
    return self.forward_pass(encoder_output, htilde_tm1, decoder_input)

  def forward_pass(self, encoder_output, htilde_tm1, decoder_input):
    """
    Inputs:
      encoder_output: encoder output of size (batch_size, self.seq_len, self.hidden_state_size)
      htilde_tm1: tensors corresponding to previous hidden state of size (batch_size, self.hidden_state_size)
                  For LSTM, this is tuple of 2 tensors
      decoder_input: the input to the decoder of size (batch_size, self.seq_len, self.output_size)
            
    Outputs:
      htilde_t: hidden state at t = self.seq_len with size of (batch_size, self.hidden_state_size) or tuple of 2 tensors for LSTM
    """
    
    if self.cell_type == 'LSTM':
        hidden = (htilde_tm1[0], torch.zeros_like(htilde_tm1[0]))
    else:
        hidden = htilde_tm1
    output, _ = self.rnn(decoder_input, hidden)
    
    return output
    
  def get_first_hidden_state(self, encoder_output):
    """Get the first hidden state to the decoder. 
    Return: First layer is the concatenation of last encoder hidden state in forward and backward directions.
            the rest of the layers are zeros.
            shape: (num_layers x batch_size x self.hidden_state_size)
    """
    ## Note: only works with BidirectionalEncoderWithAttention
    h_forward = encoder_output[:, -1, :self.hidden_state_size//2] ## (batch_size, encoder_hidden_state_size)
    h_backward = encoder_output[:, 0, self.hidden_state_size//2:] ## (batch_size, encoder_hidden_state_size)
    htilde_tm1 = torch.cat((h_forward, h_backward), dim=1).unsqueeze(0) ## (1, batch_size, encoder_hidden_state_size*2)
    htilde_tm1_rest = torch.zeros((self.num_hidden_layers - 1, encoder_output.shape[0], self.hidden_state_size), device=encoder_output.device) ## (num_layers - 1, batch_size, encoder_hidden_state_size*2)
    htilde_tm1 = torch.cat((htilde_tm1, htilde_tm1_rest), dim=0) ## (num_layers, batch_size, encoder_hidden_state_size*2)
    return htilde_tm1

## Encoder with Attention
Stacked RNN with input attention

Note: the encoder and decoder from PA3 loop over time steps because we coded the LSTM Cell (not LSTM neural net)
- Implicitly, torch.nn.LSTM is the "unrolled" neural net, whereas torch.nn.LSTMCell is a single cell (similar to what we coded in PA3)
- Since, we are applying attention to the input and that depends on the previous hidden (and cell) state(s), we must loop over each time step here
   - This goes similarly for the decoder, where we concatenate a context vector to the current input vector before applying the LSTM layer

### Unidirectional

In [ ]:
class EncoderWithAttention(nn.Module):
  def __init__(self, input_size, seq_len, num_hidden_layers, hidden_state_size, dropout=0.1, cell_type='LSTM', attn_type=None):
    super().__init__()

    self.input_size = input_size
    self.seq_len = seq_len
    self.num_hidden_layers = num_hidden_layers
    self.hidden_state_size = hidden_state_size
    self.dropout = dropout
    self.cell_type = cell_type
    self.attn_type = attn_type
    self.init_submodules()


  def init_submodules(self):
    """Initialize the submodules for encoder with attention"""
    
    if self.cell_type == "VanillaRNN":
      net = [nn.RNNCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.RNNCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    elif self.cell_type == "LSTM":
      net = [nn.LSTMCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.LSTMCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    
    else:
      net = [nn.GRUCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.GRUCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    
    self.rnn = nn.Sequential(*net)

    self.project_to_input_dim = nn.Linear(self.hidden_state_size, self.input_size)

    if self.attn_type == "Additive":
      attn = [AdditiveAttention(hidden_size=self.seq_len, encoder=True) for _ in range(self.num_hidden_layers)]
    elif self.attn_type == "Scaled-Dot":
      attn = [ScaledDotAttention(hidden_size=self.seq_len, hidden_size2=self.input_size, encoder=True)]
      attn.extend([ScaledDotAttention(hidden_size=self.seq_len, hidden_size2 = self.hidden_state_size, encoder=True) for _ in range(self.num_hidden_layers - 1)])
    elif self.attn_type == "Linear":
      attn = [LinearAttention(hidden_size=self.seq_len, encoder=True) for _ in range(self.num_hidden_layers)]
    else:
      attn = [DaRnnAttention(hidden_size=self.seq_len) for _ in range(self.num_hidden_layers)]

    self.attn = nn.Sequential(*attn)


  def forward(self, encoder_input):
    """Forward pass of encoder with attention"""
    
    attentions_all_layers = []

    ## Iterate over all layers
    for i in range(self.num_hidden_layers):
      attentions = []
      
      ## Get first hidden state
      h_prev = self.get_first_hidden_state(encoder_input) ## (batch_size, input_size) or (batch_size, hidden_size) 

      ## Loop over the sequence
      hiddens = []
      for j in range(self.seq_len):
        current_input = encoder_input[:, j, :] ## Get input at current time step (batch_size, input_size) or (batch_size, hidden_size)
        encoder_input = encoder_input.transpose(1, 2) ## (batch_size, input_size, seq_len) or (batch_size, hidden_size, seq_len)

        if self.cell_type == "LSTM":
            if h_prev[0].shape[1] != encoder_input.shape[1]:
                h_prev_pr = self.project_to_input_dim(h_prev[0]) ## (batch_size, input_size)
            else:
                h_prev_pr = h_prev[0]
        else: 
            if h_prev.shape[1] != encoder_input.shape[1]:
                h_prev_pr = self.project_to_input_dim(h_prev)
            else:
                h_prev_pr = h_prev

        h_repeated = h_prev_pr.unsqueeze(-1).expand_as(encoder_input)

        ## Pass through attention layer
        _, attention_weights = self.attn[i](h_repeated, encoder_input, encoder_input) ## (batch_size, input_size, 1) or (batch_size, hidden_size, 1)
        ## Pass through rnn
        weighted_input = attention_weights.squeeze(2) * current_input
        if i == 0:
          h_prev = self.rnn[i](weighted_input) ## First hidden = 0
        else:
          h_prev = self.rnn[i](weighted_input, h_prev) ## (batch_size, hidden_size)
        
        ## Keep important stuff in arrays
        attentions.append(attention_weights) ## (batch_size, hidden_size) 
        if self.cell_type == "LSTM":
          hiddens.append(h_prev[0]) ## (batch_size, hidden_size) 
        else:
          hiddens.append(h_prev)

        ## Revert back to previous orientation
        encoder_input = encoder_input.transpose(1, 2) ## (batch_size, seq_len, input_size) or (batch_size, seq_len, hidden_size)
      
      ## outputs of previous layer are inputs to the next layer
      encoder_input = torch.stack(hiddens, dim=1) ## (batch_size, seq_len, hidden_size)
      attentions = torch.cat(attentions, dim=2) ## (batch_size, hidden_size, seq_len)
      attentions_all_layers.append(attentions)
    
    all_hiddens = encoder_input
    ## attentions_all_layers is a list of num_layers tensors of size (batch_size, input or hidden_size, seq_len)
    return all_hiddens, attentions_all_layers

  def get_first_hidden_state(self, encoder_input):
    ## Just return tensor of zeros or tuple if LSTM
    ## Return shape: (batch_size, hidden_size)
    htilde_tm1 = torch.zeros((encoder_input.shape[0], self.hidden_state_size), device=encoder_input.device)
    if self.cell_type == "LSTM":
      return htilde_tm1, torch.zeros_like(htilde_tm1)
    else:
      return htilde_tm1

### Bidirectional

In [ ]:
class BidirectionalEncoderWithAttention(nn.Module):
    def __init__(self, input_size, seq_len, num_hidden_layers, hidden_state_size, dropout=0.1, cell_type='LSTM', attn_type=None):
      super().__init__()

      self.input_size = input_size
      self.seq_len = seq_len
      self.num_hidden_layers = num_hidden_layers
      self.hidden_state_size = hidden_state_size
      self.dropout = dropout
      self.cell_type = cell_type
      self.attn_type = attn_type
      self.init_submodules()

    def init_submodules(self):
      """Initialize the submodules for encoder with attention"""
      
      if self.cell_type == "VanillaRNN":
        net = [nn.RNNCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
        net.extend([nn.RNNCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                    for _ in range(self.num_hidden_layers - 1)])
      elif self.cell_type == "LSTM":
        net = [nn.LSTMCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
        net.extend([nn.LSTMCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                    for _ in range(self.num_hidden_layers - 1)])
      
      else:
        net = [nn.GRUCell(input_size=self.input_size, hidden_size=self.hidden_state_size)]
        net.extend([nn.GRUCell(input_size=self.hidden_state_size, hidden_size=self.hidden_state_size)
                                    for _ in range(self.num_hidden_layers - 1)])
      
      self.rnn_forward = nn.Sequential(*net)
      self.rnn_backward = nn.Sequential(*net.copy())

      self.project_to_input_dim_forward = nn.Linear(self.hidden_state_size, self.input_size)
      self.project_to_input_dim_backward = nn.Linear(self.hidden_state_size, self.input_size)

      if self.attn_type == "Additive":
        attn = [AdditiveAttention(hidden_size=self.seq_len, encoder=True) for _ in range(self.num_hidden_layers)]
      elif self.attn_type == "Scaled-Dot":
        attn = [ScaledDotAttention(hidden_size=self.seq_len, hidden_size2=self.input_size, encoder=True)]
        attn.extend([ScaledDotAttention(hidden_size=self.seq_len, hidden_size2 = self.hidden_state_size, encoder=True) for _ in range(self.num_hidden_layers - 1)])
      elif self.attn_type == "Linear":
        attn = [LinearAttention(hidden_size=self.seq_len, encoder=True) for _ in range(self.num_hidden_layers)]
      else:
        attn = [DaRnnAttention(hidden_size=self.seq_len) for _ in range(self.num_hidden_layers)]

      self.attn_forward = nn.Sequential(*attn)
      self.attn_backward = nn.Sequential(*attn.copy()) 
      
      self.forward_attn_weights = None
      self.backward_attn_weights = None

    def forward(self, data):
      """Forward pass of data in encoder with input attention"""

      forward_output, self.forward_attn_weights = self.forward_pass(data)
      backward_output, self.backward_attn_weights = self.backward_pass(data)
      output = torch.cat([forward_output, backward_output], dim=2)
      
      return output, (self.forward_attn_weights, self.backward_attn_weights)

    def forward_pass(self, data, direction='Forward'):
      """Forward pass of encoder with attention in the forward direction"""
      
      attentions_all_layers = []

      if direction == 'Forward':
          attn_net = self.attn_forward
          rnn = self.rnn_forward
          project_to_input_dim = self.project_to_input_dim_forward
      else:
          attn_net = self.attn_backward
          rnn = self.rnn_backward
          project_to_input_dim = self.project_to_input_dim_backward

      ## Iterate over all layers
      for i in range(self.num_hidden_layers):
        attentions = []
        
        ## Get first hidden state
        h_prev = self.get_first_hidden_state(data) ## (batch_size, input_size) or (batch_size, hidden_size) 

        ## Loop over the sequence
        hiddens = []
        for j in range(self.seq_len):
          current_input = data[:, j, :] ## Get input at current time step (batch_size, input_size) or (batch_size, hidden_size)
          data = data.transpose(1, 2) ## (batch_size, input_size, seq_len) or (batch_size, hidden_size, seq_len)

          if self.cell_type == "LSTM":
              if h_prev[0].shape[1] != data.shape[1]:
                  h_prev_pr = project_to_input_dim(h_prev[0]) ## (batch_size, input_size)
              else:
                  h_prev_pr = h_prev[0]
          else:
              if h_prev.shape[1] != data.shape[1]:
                  h_prev_pr = project_to_input_dim(h_prev)
              else:
                  h_prev_pr = h_prev

          h_repeated = h_prev_pr.unsqueeze(-1).expand_as(data)  ## (batch_size, input_size, seq_len)

          ## Pass through attention layer
          _, attention_weights = attn_net[i](h_repeated, data, data) ## (batch_size, input_size, 1) or (batch_size, hidden_size, 1)
          ## Pass through rnn
          weighted_input = attention_weights.squeeze(2) * current_input
          h_prev = rnn[i](weighted_input, h_prev) ## (batch_size, hidden_size)
          
          ## Keep important stuff in arrays
          attentions.append(attention_weights) ## (batch_size, hidden_size) 
          if self.cell_type == "LSTM":
            hiddens.append(h_prev[0]) ## (batch_size, hidden_size) 
          else:
            hiddens.append(h_prev)

          ## Revert back to previous orientation
          data = data.transpose(1, 2) ## (batch_size, seq_len, input_size) or (batch_size, seq_len, hidden_size)
        
        ## outputs of previous layer are inputs to the next layer
        data = torch.stack(hiddens, dim=1) ## (batch_size, seq_len, hidden_size)
        attentions = torch.cat(attentions, dim=2) ## (batch_size, hidden_size, seq_len)
        attentions_all_layers.append(attentions)
      
      final_hiddens = data
      ## attentions_all_layers is a list of num_layers tensors of size (batch_size, input or hidden_size, seq_len)
      return final_hiddens, attentions_all_layers
    
    def backward_pass(self, data):
        """RNN in the reverse direction"""
        ## Reverse the input data along the sequence dimension
        data = data.flip(dims=(1,))

        return self.forward_pass(data, direction='Backward')

    def get_first_hidden_state(self, encoder_input):
      ## Just return tensor of zeros or tuple if LSTM
      ## Return shape: (batch_size, hidden_size)
      htilde_tm1 = torch.zeros((encoder_input.shape[0], self.hidden_state_size), device=encoder_input.device)
      if self.cell_type == "LSTM":
        return htilde_tm1, torch.zeros_like(htilde_tm1)
      else:
        return htilde_tm1

## Decoder with Attention
Stacked RNN with attention

In [ ]:
class DecoderWithAttention(nn.Module):
  def __init__(self, output_size, seq_len, num_hidden_layers, hidden_state_size, cell_type='LSTM', attn_type='Additive'):
    super().__init__()

    self.output_size = output_size
    self.seq_len = seq_len
    self.num_hidden_layers = num_hidden_layers
    self.hidden_state_size = hidden_state_size 
    self.cell_type = cell_type
    self.attn_type = attn_type
    self.init_submodules()


  def init_submodules(self):
    """Initialize the submodules for decoder with attention"""

    if self.cell_type == "VanillaRNN":
      net = [nn.RNNCell(input_size=self.output_size + self.hidden_state_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.RNNCell(input_size=self.hidden_state_size*2, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    elif self.cell_type == "LSTM":
      net = [nn.LSTMCell(input_size=self.output_size + self.hidden_state_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.LSTMCell(input_size=self.hidden_state_size * 2, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    else:
      net = [nn.GRUCell(input_size=self.output_size + self.hidden_state_size, hidden_size=self.hidden_state_size)]
      net.extend([nn.GRUCell(input_size=self.hidden_state_size * 2, hidden_size=self.hidden_state_size)
                                  for _ in range(self.num_hidden_layers - 1)])
    
    self.rnn = nn.Sequential(*net)

    if self.attn_type == "Additive":
      attn = [AdditiveAttention(hidden_size=self.hidden_state_size) for _ in range(self.num_hidden_layers)]
    elif self.attn_type == "Scaled-Dot":
      attn = [ScaledDotAttention(hidden_size=self.hidden_state_size) for _ in range(self.num_hidden_layers)]
    elif self.attn_type == "Linear":
      attn = [LinearAttention(hidden_size=self.hidden_state_size) for _ in range(self.num_hidden_layers)]
    else:
      attn = [DaRnnAttention(hidden_size=self.hidden_state_size) for _ in range(self.num_hidden_layers)]
  
    self.attn = nn.Sequential(*attn)

  def forward(self, encoder_output, decoder_input):
    htilde_tm1 = self.get_first_hidden_state(encoder_output)
    return self.forward_pass(encoder_output, htilde_tm1, decoder_input)


  def forward_pass(self, encoder_output, htilde_tm1, decoder_input):
    """Forward pass of decoder with attention.
    Params:
    encoder_output: output of the encoder (batch_size, seq_len, hidden_size)
                    Note: hidden_size = encoder_hidden_size * 2 (if encoder is bidirection; otherwise, equal)
    htilde_tm1: first hidden state (tensor or tuple of 2 tensors) (batch_size, hidden_size)
    decoder_input: input to the decoder (batch_size, seq_len, output_size)
    """

    attentions_all_layers = []

    ## Iterate over all layers
    for i in range(self.num_hidden_layers):
      attentions = []
      ## Get first hidden state for the layer
      h_prev = htilde_tm1

      ## Loop over the sequence
      hiddens = []
      for j in range(self.seq_len):
        current_input = decoder_input[:, j, :] ## Get input at current time step (batch_size, output_size)
        ## Pass through attention layer
        if self.cell_type == "LSTM":
          context, attention_weights = self.attn[i](h_prev[0], encoder_output, encoder_output) ## (batch_size, 1, hidden_size)
        else:
          context, attention_weights = self.attn[i](h_prev, encoder_output, encoder_output)
        
        ## Concatenate with the current inputs
        input_and_context = torch.cat([current_input, context.squeeze(1)], dim=1) ## (batch_size, hidden_size + output_size)
        h_prev = self.rnn[i](input_and_context, h_prev) ## (batch_size, hidden_size)

        attentions.append(attention_weights) ## (batch_size, seq_len, 1)
        if self.cell_type == "LSTM":
          hiddens.append(h_prev[0]) ## (batch_size, hidden_size)
        else:
          hiddens.append(h_prev)
      
      ## outputs of previous layer are inputs to the next layer
      decoder_input = torch.stack(hiddens, dim=1) ## (batch_size, seq_len, hidden_size)
      attentions = torch.cat(attentions, dim=2) ## (batch_size, seq_len, seq_len)
      attentions_all_layers.append(attentions)
    
    final_hidden = decoder_input[:, -1, :]
    all_hiddens = decoder_input
    attentions_all_layers = torch.stack(attentions_all_layers, dim=0) ## (num_layers, batch_size, seq_len, seq_len)
    return all_hiddens, final_hidden, attentions_all_layers

  def get_first_hidden_state(self, encoder_output):
    ## Just return tensor of zeros
    ## Return shape: (batch_size, hidden_size)
    batch_size = encoder_output.shape[0]
    htilde_tm1 = torch.zeros((batch_size, self.hidden_state_size), device=encoder_output.device)
    if self.cell_type == 'LSTM':
        htilde_tm1 = htilde_tm1, torch.zeros_like(htilde_tm1)
    return htilde_tm1

# Attention Layers

## Additive Attention

In [ ]:
class AdditiveAttention(nn.Module):
  def __init__(self, hidden_size, encoder=False):
    super().__init__()

    self.hidden_size = hidden_size
    self.encoder = encoder
    self.init_submodules()

  def init_submodules(self):
    self.attention = nn.Sequential(nn.Linear(2 * self.hidden_size, self.hidden_size),
                                   nn.ReLU(),
                                   nn.Linear(self.hidden_size, 1))

    self.softmax = nn.Softmax(dim=1)


  def forward(self, queries, keys, values):
    """
    Inputs:
      queries: current decoder hidden state (batch_size, hidden_size)
        if self.encoder: previous encoder hidden state (batch_size, input_size/hidden_size, seq_len)
      keys: all encoder hidden states  (batch_size, seq_len, hidden_size)
        if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
      values: all encoder hidden states (batch_size, seq_len, hidden_size)
        if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
    Outputs:
      context: weighted average of the keys/values (for decoder only) (batch_size, 1, hidden_size)
      attn_weights: calculated weights (batch_size, seq_len, 1)
        if self.encoder: (batch_size, input_size/hidden_size, 1)
    """
    batch_size = keys.size(0)
    if not self.encoder:
      expanded_queries = queries.view(batch_size, -1, self.hidden_size).expand_as(keys)
    else:
      expanded_queries = queries
    concat_input = torch.cat((expanded_queries, keys), dim=2)
    unnormalized_attn_weights = self.attention(concat_input)
    attn_weights = self.softmax(unnormalized_attn_weights)  # normalize weights
    context = torch.bmm(attn_weights.transpose(2,1), values)

    return context, attn_weights

## (Modified) Scaled Dot Product Attention

In [ ]:
class ScaledDotAttention(nn.Module):
    def __init__(self, hidden_size, hidden_size2=0, encoder=False):
        """Note that if encoder = True, hidden_size = seq_len, hidden_size2 is actual hidden size of encoder
        """
        super().__init__()

        self.encoder = encoder
        self.hidden_size = hidden_size
        if self.encoder:
            self.hidden_size2 = hidden_size2
        else:
            self.hidden_size2 = hidden_size
        self.init_submodules()

    def init_submodules(self):
        self.Q = nn.Linear(self.hidden_size, self.hidden_size)
        self.K = nn.Linear(self.hidden_size, self.hidden_size)
        self.V = nn.Linear(self.hidden_size, self.hidden_size)
        self.softmax = nn.Softmax(dim=1)
        self.scaling_factor = torch.rsqrt(torch.tensor(self.hidden_size, dtype= torch.float))

        self.lin = nn.Linear(self.hidden_size2, 1)
    
    def forward(self, queries, keys, values):
      """
      Inputs:
        queries: current decoder hidden state (batch_size, hidden_size)
          if self.encoder: previous encoder hidden state (batch_size, input_size/hidden_size, seq_len)
        keys: all encoder hidden states  (batch_size, seq_len, hidden_size)
          if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
        values: all encoder hidden states (batch_size, seq_len, hidden_size)
          if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
      Outputs:
        context: weighted average of the keys/values (for decoder only) (batch_size, 1, hidden_size)
        attn_weights: calculated weights (batch_size, seq_len, 1)
          if self.encoder: (batch_size, input_size/hidden_size, 1)
      """
      batch_size = queries.shape[0]
      if not self.encoder:
        q = self.Q(queries).view(batch_size, -1, self.hidden_size).transpose(2, 1)
      else:
        q = queries.transpose(2, 1)
      k = self.K(keys)
      v = self.V(values)
      unnormalized_attention = (k @ q) * self.scaling_factor # (B, S, H) x (B, H, k) = (B, S, k)
      if self.encoder:
        unnormalized_attention = self.lin(unnormalized_attention) # (B, S, 1)
      attention_weights = self.softmax(unnormalized_attention) # (B, S, k) or (B, S, 1) if encoder
      if self.encoder:
        context = None
      else:
        context = (attention_weights.transpose(2,1) @ v).squeeze() # (B, k, S) x (B, S, H) = (B, k, H)
      return context, attention_weights

## Linear Attention

In [ ]:
class LinearAttention(nn.Module):
    def __init__(self, hidden_size, encoder=False):
        super().__init__()

        self.hidden_size = hidden_size
        self.encoder = encoder
        self.init_submodules()


    def init_submodules(self):
        self.attn = nn.Linear(self.hidden_size * 2, 1)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, queries, keys, values):
      """
      Inputs:
        queries: current decoder hidden state (batch_size, hidden_size)
          if self.encoder: previous encoder hidden state (batch_size, input_size/hidden_size, seq_len)
        keys: all encoder hidden states  (batch_size, seq_len, hidden_size)
          if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
        values: all encoder hidden states (batch_size, seq_len, hidden_size)
          if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
      Outputs:
        context: weighted average of the keys/values (for decoder only) (batch_size, 1, hidden_size)
        attn_weights: calculated weights (batch_size, seq_len, 1)
          if self.encoder: (batch_size, input_size/hidden_size, 1)
      """
      if not self.encoder:
          queries = queries.view((-1, 1, self.hidden_size)).expand_as(keys)
      cat = torch.cat([queries, keys], dim=2) ## (batch_size, seq_len, hidden_size * 2)
      unnormalized_attention = self.attn(cat) ## (batch_size, seq_len, 1)
      attention_weights = self.softmax(unnormalized_attention)
      context = None
      if not self.encoder:
          context = (attention_weights.transpose(2,1) @ values).squeeze() # (B, 1, S) x (B, S, H) = (B, 1, H)
      return context, attention_weights

## DA-RNN Attention
Note: same thing as Additive Attention, but with Tanh activation

In [ ]:
class DaRnnAttention(nn.Module):
  def __init__(self, hidden_size, encoder=False):
    super().__init__()

    self.hidden_size = hidden_size
    self.encoder = encoder
    self.init_submodules()


  def init_submodules(self):
    self.attention = nn.Sequential(nn.Linear(2*self.hidden_size, self.hidden_size),
                                   nn.Tanh(),
                                   nn.Linear(self.hidden_size, 1))

    self.softmax = nn.Softmax(dim=1)


  def forward(self, queries, keys, values):
    """
    Inputs:
      queries: current decoder hidden state (batch_size, hidden_size)
        if self.encoder: previous encoder hidden state (batch_size, input_size/hidden_size, seq_len)
      keys: all encoder hidden states  (batch_size, seq_len, hidden_size)
        if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
      values: all encoder hidden states (batch_size, seq_len, hidden_size)
        if self.encoder: all inputs (batch_size, input_size/hidden_size, seq_len)
    Outputs:
      context: weighted average of the keys/values (for decoder only) (batch_size, 1, hidden_size)
      attn_weights: calculated weights (batch_size, seq_len, 1)
        if self.encoder: (batch_size, input_size/hidden_size, 1)
    """
    batch_size = keys.size(0)
    if not self.encoder:
      expanded_queries = queries.view(batch_size, -1, self.hidden_size).expand_as(keys)
    else:
      expanded_queries = queries
    concat_input = torch.cat((expanded_queries, keys), dim=2)
    unnormalized_attn_weights = self.attention(concat_input)
    attn_weights = self.softmax(unnormalized_attn_weights)  # normalize weights
    context = torch.bmm(attn_weights.transpose(2,1), values)

    return context, attn_weights

# Training and Evaluation

In [ ]:
## Training and Evaluation Code 
## Note: for optimization, we use MSE loss

def compute_loss(output, target, loss_fn):
  """
  Arguments:
    model: type of model being used
    data: input data we compute loss on. Make sure to split data properly depending on the type of model being used 'R' vs 'ED'
      For 'R' we just need the main features (X_tilde and y), for 'ED' we set X for encoder_input, X_tilde for decoder_input and y as expected output
      Maybe for 'R' we use all features?
    loss_fn: the loss function being used 
  """
  loss = loss_fn(output, target) 

  return loss


def training_loop(model, dataloader, optimizer, device):
    """Training loop
    Params:
    model: the model
    dataloader: the training data
    Optimizer: optimizer for gradient descent
    device: cpu or cuda
    """
    loss_fn = nn.MSELoss().to(device) ## average MSE loss
    total_train_loss = 0
    total_train_sequences = 0

    for i, data in enumerate(dataloader): 
        # print("Batch {}".format(i))
        X_train, X_tilde_train, y_train = data

        ## Send to device
        X_train = X_train.to(device=device) 
        if X_tilde_train is not None:
            X_tilde_train = X_tilde_train.to(device=device) 
        y_train = y_train.to(device=device)

        # Zero out gradients
        optimizer.zero_grad()

        # Run input through model
        output = model(X_train, X_tilde_train)

        # Calculate batched training loss
        loss = compute_loss(output, y_train, loss_fn)
        num_sequences = X_train.shape[0]
        total_train_loss += loss * num_sequences
        total_train_sequences += num_sequences
        
        # Backward pass
        loss.backward()
        # Update gradients
        optimizer.step()

        del X_train, X_tilde_train, y_train, loss ## To save space

    return total_train_loss / total_train_sequences


def train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, opts):
    """Train model
    Params:
    X_train: input to encoder
    X_tilde_train: input to decoder
    y_train: training labels
    X_val: validation input to encoder
    X_tilde_val: validation input to decoder
    y_val: training labels
    opts: dictionary of hyperparameters (AttrDict)
    """

    loss_fn = nn.MSELoss().to(opts.device)
    dataloader = CovidDataLoader(X_train, X_tilde_train, y_train, opts.batch_size)
    if opts.model == 'ED':
      model = EncoderDecoder(opts.encoder_class, opts.decoder_class, opts.input_size, opts.seq_len, opts.encoder_num_hidden_layers, 
               opts.hidden_state_size, opts.output_size, opts.dropout, opts.cell_type,
               opts.enc_attn_type, opts.decoder_num_hidden_layers, opts.dec_attn_type, opts.bidirectional, opts.temporal_attn)
    else:
      model = MyRNN(opts.rnn_class, opts.input_size, opts.hidden_state_size, opts.output_size, opts.num_layers, opts.seq_len, opts.dropout, 
                    opts.bidirectional, opts.cell_type, opts.rnn_attn_type, opts.temporal_attn)

    if opts.device == 'cuda':
        model.cuda()

    optimizer = optim.Adam(list(model.parameters()), lr=opts.lr) 

    losses = {'epoch': [], 'train': [], 'val': []}
    X_val = X_val.type(torch.FloatTensor).to(device=opts.device)
    if X_tilde_val is not None:
        X_tilde_val = X_tilde_val.type(torch.FloatTensor).to(device=opts.device)
    y_val = y_val.type(torch.FloatTensor).to(device=opts.device)

    mae_fn = nn.L1Loss().to(opts.device)

    best_val_loss = float('inf')
    early_stopping_counter = 0
    for epoch in range(opts.num_epochs):
        ## Update params
        avg_train_loss = training_loop(model, dataloader, optimizer, opts.device)

        ## Calculate validation loss
        output = model(X_val, X_tilde_val) 
        avg_val_loss = compute_loss(output, y_val, loss_fn)
        avg_mae_loss = mae_fn(output, y_val)

        if avg_val_loss < best_val_loss:
            checkpoint(model, opts)
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
        
        if early_stopping_counter > opts.early_stopping_patience:
            # print("Validation loss has not improved in {} epochs, stopping early".format(opts.early_stopping_patience))
            # print("Obtained lowest validation loss of: {}".format(best_val_loss))
            return losses

        losses['epoch'].append(epoch)
        losses['train'].append(avg_train_loss)
        losses['val'].append(avg_val_loss)
        
        if epoch > 0 and avg_val_loss > losses['val'][-2]: ## lr decay
                optimizer.param_groups[0]['lr'] *= opts.lr_decay

        if opts.print:
            print("Epoch {} | Train loss: {:.3f} | Val loss: {:.3f} | Val MAE: {:.3f}".format(epoch, avg_train_loss, avg_val_loss, avg_mae_loss))
    return losses

def checkpoint(model, opts):
    """Saves the current model.
    """
    with open('{}.pt'.format(opts.name), 'wb') as f:
        torch.save(model, f)

# Visualize Attention

In [ ]:
def get_condition_window(opts, num=None):
    """Get the dates for the condition window"""
    if num is None:
        pred_date_idx = np.where(ALL_DATES == opts.pred_date)[0][0]
    else:
        pred_date_idx = np.where(ALL_DATES == opts.pred_date[num])[0][0]
    
    cond_start = pred_date_idx - opts.k - opts.t + 1
    condition_window = ALL_DATES[cond_start : cond_start + opts.k]

    return condition_window

def visualize_attention_helper(attention, opts, fig, ax, encoder=False, forward=True): 
  """
  Inputs:
    attention: encoder or decoder attention matrices
    opts: plotting options to customize output figure and subplots
    encoder: whether visualization is for encoder or decoder
  """ 
  if not encoder:
      batch_size, hidden_size, seq_len = attention.shape  # hidden_size = seq_len if decoder, otherwise for encoder
  else:
      batch_size, hidden_size, seq_len = attention[0].shape
  
  if isinstance(opts.plot_num, int):
      assert opts.plot_num <= batch_size - 1
  else:
      assert opts.plot_num[-1] <= batch_size - 1

  if not encoder:
      attention = attention.cpu().detach().numpy() 
  else:
      attention = attention[0].cpu().detach().numpy()
  
  if isinstance(opts.plot_num, int):
      opts.plot_num = np.array([opts.plot_num])
      opts.pred_date = np.arrage([opts.pred_date])

  for i, plot_num in enumerate(opts.plot_num):
      layer_attn = attention[plot_num, :, :]  # attention of single layer (seq_len, hidden_size or seq_len)

      cax = ax[i].matshow(layer_attn, vmin=0., vmax=1.)

      condition_window = get_condition_window(opts, num=i)
      if encoder:
          ax[i].set_yticklabels([''] + opts.in_features)

          if forward:
              x_label = "forward input, {}".format(opts.pred_date[i])
          else:
              x_label = "backward input, {}".format(opts.pred_date[i])

          ax[i].set_xticklabels([''] + list(condition_window), rotation=90)
      elif opts.plt_temporal and not encoder:
          ax[i].set_yticklabels([''] + list(condition_window))
          ax[i].set_xticklabels([''])
          x_label = "temporal, {}".format(opts.pred_date[i])

      ax[i].xaxis.set_major_locator(ticker.MultipleLocator(1))
      ax[i].yaxis.set_major_locator(ticker.MultipleLocator(1))
      
      ax[i].grid('off')
      ax[i].set_xlabel(x_label)


def visualize_attention(model, plt_opts):
  """
  Inputs:
    model: final model after training
    plt_opts: plotting options to customize output figure and subplots
  
  Shows attention for encoder and decoder if it exists
  """
  
  if isinstance(plt_opts.plot_num, np.ndarray):
      ncols = plt_opts.plot_num.shape[0]
  else:
      ncols = 1

  if plt_opts.model_type == 'ed':
      input_attn = model.input_attn_weights
      if input_attn is not None:
          forward_attn, backward_attn = input_attn
      else:
          forward_attn, backward_attn = None, None
      temporal_attn = model.temporal_attn_weights 
  else:
      forward_attn, backward_attn, temporal_attn = model.forward_attn_weights, model.backward_attn_weights, model.temporal_attn_weights

  ## Visualize input attention weights if they are not None
  if forward_attn is not None and plt_opts.plt_forward:
      fig, ax = plt.subplots(nrows=1, ncols=ncols, figsize=(plt_opts.height, plt_opts.width), constrained_layout=True) 
      visualize_attention_helper(forward_attn, plt_opts, fig, ax, encoder=True)
      plt.savefig("experiment3/{}_{}.pdf".format(plt_opts.model_name, "forward"))
      plt.show()

  if backward_attn is not None and plt_opts.plt_backward:
      fig, ax = plt.subplots(nrows=1, ncols=ncols, figsize=(plt_opts.height, plt_opts.width), constrained_layout=True) 
      visualize_attention_helper(backward_attn, plt_opts, fig, ax, encoder=True, forward=False)
      plt.savefig("experiment3/{}_{}.pdf".format(plt_opts.model_name, "backward"))
      plt.show()
  # visualize the temporal attention weights if it is not None
  if temporal_attn is not None and plt_opts.plt_temporal:
      fig, ax = plt.subplots(nrows=1, ncols=ncols, figsize=(plt_opts.height, plt_opts.width), constrained_layout=True) 
      visualize_attention_helper(temporal_attn, plt_opts, fig, ax, encoder=False)
      plt.savefig("experiment4/{}_{}.pdf".format(plt_opts.model_name, "temporal"))
      plt.show()

# Run Models

## Run RNN

In [ ]:
## Loop over every subset of in_features
opts = AttrDict()
args_dict = {       'name': 'rnn_wo_attn',
                    'model':'R', ## R, ED
                    'rnn_class': RNNWithoutAttention,
                    'rnn_attn_type': None,
                    'temporal_attn': True,
                    'input_size': 15, ## Random -- to be set in the loop
                    'output_size': 3, ## Random
                    'seq_len': 5, ## Random
                    'hidden_state_size': 16,
                    'num_layers': 1, 
                    'bidirectional': True, 
                    'dropout': 0.,
                    'cell_type': 'LSTM', ## VanillaRNN, LSTM, GRU
                    'device': 'cpu', ## 'cuda', 'cpu'
                    'num_epochs': 70, 
                    'rnn_attn_type': None,
                    'lr':1e-3,
                    'lr_decay':0.999,
                    'batch_size': 32,
                    'early_stopping_patience': 10,
                    'momentum': 0.9,
                    'print': True
      }
opts.update(args_dict)


In [ ]:
out_features = ['Deaths']
in_features = BASIC_FEATURES + TEST_FEATURES + HOSPITAL_FEATURES + LTC_FEATURES
X, X_tilde, y, y_cumul, dates, in_features = cluster_data(df, k=5, t=7, separate=True, in_features=in_features, out_features=out_features, moving_average=True, m=3)
X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test = train_val_test_split(X, X_tilde, y, dates)

opts.input_size = X_train.shape[2]
opts.output_size = y_train.shape[1]
opts.seq_len = X_train.shape[1]
losses = train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, opts)

In [ ]:
trained_rnn = torch.load('rnn_wo_attn.pt')
args_dict = opts
pred = trained_rnn(X_train.to(args_dict.device), X_tilde_train.to(args_dict.device))
pred = pred.cpu().detach().numpy()
plt.figure()
plt.plot(y_train, label="truth")
plt.plot(pred, label="pred")
plt.title("Cumulative Training Deaths")
plt.legend()
plt.show()

X_train_last_day_of_condition = X_tilde_train[:, -1, 0].cpu().detach().numpy()
pred_diff = pred[:, 0] - X_train_last_day_of_condition
y_train_diff = y_train[:, 0] - X_train_last_day_of_condition
plt.figure()
plt.plot(y_train_diff, label="truth")
plt.plot(pred_diff, label="pred")
plt.title("New Training Deaths")
plt.legend()
plt.show()

pred = trained_rnn(X_val.to(args_dict.device), X_tilde_val.to(args_dict.device))
pred = pred.cpu().detach().numpy()
plt.figure()
plt.plot(y_val, label="truth")
plt.plot(pred, label="pred")
plt.title("Cumulative Validation Deaths")
plt.legend()
plt.show()

X_val_last_day_of_condition = X_tilde_val[:, -1, 0].cpu().detach().numpy()
pred_diff = pred[:, 0] - X_val_last_day_of_condition
y_val_diff = y_val[:, 0] - X_val_last_day_of_condition
plt.figure()
plt.plot(y_val_diff, label="truth")
plt.plot(pred_diff, label="pred")
plt.title("New Training Deaths")
plt.legend()
plt.show()

## Run RNN with Input Attention

In [ ]:
opts = AttrDict()
args_dict = {       'name': 'rnn_w_input_attn',
                    'model':'R', ## R, ED
                    'rnn_class': RNNWithAttention, ## RNNWithoutAttention
                    'temporal_attn': True,
                    'input_size': 15, ## Random -- to be set in the loop
                    'output_size': 3, ## Random
                    'seq_len': 5, ## Random
                    'hidden_state_size': 16,
                    'num_layers': 1, 
                    'bidirectional': True, 
                    'dropout': 0.,
                    'cell_type': 'LSTM', ## VanillaRNN, LSTM, GRU
                    'device': 'cpu', ## 'cuda', 'cpu'
                    'rnn_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None
                    'num_epochs': 100, 
                    'lr':1e-3,
                    'lr_decay':0.999,
                    'batch_size': 16,
                    'early_stopping_patience': 5,
                    'print': True
      }
opts.update(args_dict)

In [ ]:
k = 5
t = 7
out_features = ['Deaths']
in_features = BASIC_FEATURES + TEST_FEATURES + HOSPITAL_FEATURES + LTC_FEATURES
X, X_tilde, y, y_cumul, dates, in_features = cluster_data(df, k=5, t=7, separate=True, in_features=in_features, out_features=out_features, moving_average=True, m=3)
X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test = train_val_test_split(X, X_tilde, y, dates)

opts.input_size = X_train.shape[2]
opts.output_size = y_train.shape[1]
opts.seq_len = X_train.shape[1]
losses = train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, opts)

In [ ]:
model = torch.load('rnn_w_attn.pt')
model(X_train.to(opts.device), X_tilde_train.to(opts.device))
attn_plot_args = AttrDict()
plot_num = 1
plt_args = {
          'grid_on': True,
          'height': 5,
          'width': 5,
          'plot_num': plot_num,  # should be a value <= X_train.shape[0] or X_val.shape[0] and >= 1
          'layer_num': 1, # should be a value <= num layers and >= 1
          'in_features': in_features,
          'plt_forward': True,
          'plt_backward': True,
          'model_type': 'rnn',
          'k': k,
          't': t,
          'pred_date': dates[plot_num],
          'plt_temporal': True
} 
attn_plot_args.update(plt_args)
visualize_attention(model, attn_plot_args)

## Run Encoder-Decoder Architecture with Encoder and Decoder Attention

In [ ]:
opts = AttrDict()
args_dict = { 'name': 'ed_w_all_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': BidirectionalEncoderWithAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': True,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 32,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': 'Additive',
              'device': 'cpu', ## 'cuda'
              'num_epochs': 150,
              'lr':1e-3,
              'batch_size': 32,
              'lr_decay': 0.99, # or maybe 0.9
              'early_stopping_patience': 10,
              'print': True
}
opts.update(args_dict)

out_features = ['Deaths']
X, X_tilde, y, y_cumul, dates, in_features = cluster_data(df, k=5, t=7, separate=True, in_features=None, out_features=out_features, moving_average=True, m=3)
X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test = train_val_test_split(X, X_tilde, y, dates)

opts.input_size = X_train.shape[2]
opts.output_size = y_train.shape[1]
opts.seq_len = X_train.shape[1]

losses = train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, opts)

In [ ]:
model = torch.load('ed_w_attn.pt')

plt.plot(losses['epoch'], losses['val'])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

args_dict = opts
pred = model(X_train.to(args_dict.device), X_tilde_train.to(args_dict.device))
pred = pred.cpu().detach().numpy()
plt.figure()
plt.plot(y_train, label="truth")
plt.plot(pred, label="pred")
plt.title("Cumulative Training Deaths")
plt.legend()
plt.show()

X_tilde_train_last_day_of_condition = X_tilde_train[:, -1, 0].cpu().detach().numpy()
pred_diff = pred[:, 0] - X_tilde_train_last_day_of_condition
y_train_diff = y_train[:, 0] - X_tilde_train_last_day_of_condition
plt.figure()
plt.plot(y_train_diff, label="truth")
plt.plot(pred_diff, label="pred")
plt.title("New Training Deaths")
plt.legend()
plt.show()

pred = model(X_val.to(args_dict.device), X_tilde_val.to(args_dict.device))
pred = pred.cpu().detach().numpy()
plt.figure()
plt.plot(y_val, label="truth")
plt.plot(pred, label="pred")
plt.title("Cumulative Validation Deaths")
plt.legend()
plt.show()

X_tilde_val_last_day_of_condition = X_tilde_val[:, -1, 0].cpu().detach().numpy()
pred_diff = pred[:, 0] - X_tilde_val_last_day_of_condition
y_val_diff = y_val[:, 0] - X_tilde_val_last_day_of_condition
plt.figure()
plt.plot(y_val_diff, label="truth")
plt.plot(pred_diff, label="pred")
plt.title("New Training Deaths")
plt.legend()
plt.show()

## Test Attention Visualization

In [ ]:
model(X_train.to(opts.device), X_tilde_train.to(opts.device))
attn_plot_args = AttrDict()
plot_num = 2
plt_args = {
          'grid_on': True,
          'height': 5,
          'width': 5,
          'plot_num': plot_num,  # should be a value <= X_train.shape[0] or X_val.shape[0] and >= 1
          'layer_num': 1, # should be a value <= num layers and >= 1
          'in_features': in_features,
          'plt_encoder': True,
          'plt_temporal': True,
          'plt_forward': True,
          'plt_backward': True,
          'model_type': 'ed',
          'k': 5,
          't': 7,
          'pred_date': dates[plot_num]
} 
attn_plot_args.update(plt_args)
visualize_attention(model, attn_plot_args)

# Optimal Models

## RNN Without Input or Temporal Attention

In [ ]:
rnn_wo_attn_args = AttrDict()
args_dict = {       'name': 'rnn_wo_attn',
                    'model':'R', ## R, ED
                    'rnn_class': RNNWithoutAttention,
                    'rnn_attn_type': None,
                    'temporal_attn': False,
                    'input_size': 15, ## Random -- to be set in the loop
                    'output_size': 3, ## Random
                    'seq_len': 5, ## Random
                    'hidden_state_size': 16,
                    'num_layers': 1, 
                    'bidirectional': True, 
                    'dropout': 0.,
                    'cell_type': 'LSTM', ## VanillaRNN, LSTM, GRU
                    'device': 'cpu', ## 'cuda', 'cpu'
                    'num_epochs': 70, 
                    'rnn_attn_type': None,
                    'lr':1e-3,
                    'lr_decay':0.999,
                    'batch_size': 16,
                    'early_stopping_patience': 5,
                    'print': True
      }
rnn_wo_attn_args.update(args_dict)

## RNN With Input Attention

In [ ]:
rnn_w_input_attn_args = AttrDict()
args_dict = {       'name': 'rnn_w_input_attn',
                    'model':'R', ## R, ED
                    'rnn_class': RNNWithAttention, ## RNNWithoutAttention
                    'temporal_attn': False,
                    'input_size': 15, ## Random -- to be set in the loop
                    'output_size': 3, ## Random
                    'seq_len': 5, ## Random
                    'hidden_state_size': 16,
                    'num_layers': 1, 
                    'bidirectional': True, 
                    'dropout': 0.,
                    'cell_type': 'LSTM', ## VanillaRNN, LSTM, GRU
                    'device': 'cpu', ## 'cuda', 'cpu'
                    'rnn_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None
                    'num_epochs': 100, 
                    'lr':1e-3,
                    'lr_decay':0.999,
                    'batch_size': 16,
                    'early_stopping_patience': 5,
                    'print': True
      }
rnn_w_input_attn_args.update(args_dict)

## RNN With Temporal Attention

In [ ]:
rnn_w_temp_attn_args = AttrDict()
args_dict = {       'name': 'rnn_w_temp_attn',
                    'model':'R', ## R, ED
                    'rnn_class': RNNWithoutAttention, ## RNNWithoutAttention
                    'temporal_attn': True,
                    'input_size': 15, ## Random -- to be set in the loop
                    'output_size': 3, ## Random
                    'seq_len': 5, ## Random
                    'hidden_state_size': 16,
                    'num_layers': 1, 
                    'bidirectional': True, 
                    'dropout': 0.,
                    'cell_type': 'LSTM', ## VanillaRNN, LSTM, GRU
                    'device': 'cpu', ## 'cuda', 'cpu'
                    'rnn_attn_type': None, ## Additive, Scaled-Dot, Linear, DA-RNN, None
                    'num_epochs': 100, 
                    'lr':1e-3,
                    'lr_decay':0.999,
                    'batch_size': 32,
                    'early_stopping_patience': 5,
                    'print': True
      }
rnn_w_temp_attn_args.update(args_dict)

## RNN With Input and Temporal Attention

In [ ]:
rnn_w_attn_args = AttrDict()
args_dict = {       'name': 'rnn_w_attn',
                    'model':'R', ## R, ED
                    'rnn_class': RNNWithAttention, ## RNNWithoutAttention
                    'temporal_attn': True,
                    'input_size': 15, ## Random -- to be set in the loop
                    'output_size': 3, ## Random
                    'seq_len': 5, ## Random
                    'hidden_state_size': 16,
                    'num_layers': 1, 
                    'bidirectional': True, 
                    'dropout': 0.,
                    'cell_type': 'LSTM', ## VanillaRNN, LSTM, GRU
                    'device': 'cpu', ## 'cuda', 'cpu'
                    'rnn_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None
                    'num_epochs': 100, 
                    'lr':1e-3,
                    'lr_decay':0.999,
                    'batch_size': 16,
                    'early_stopping_patience': 5,
                    'print': True
      }
rnn_w_attn_args.update(args_dict)

## EncoderDecoder with no Attention

In [ ]:
# this one is a bit noisy. I'm trying to make it more consistent still
ed_wo_attn_args = AttrDict()
args_dict = { 'name': 'ed_wo_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': EncoderWithoutAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithoutAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': False,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 20, #12,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': None, ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': None,
              'device': 'cpu', ## 'cuda'
              'num_epochs': 300,
              'lr':1e-3,
              'batch_size': 22, #32,
              'lr_decay': 0.8,
              'early_stopping_patience': 20,
              'print': True
}
ed_wo_attn_args.update(args_dict)


## EncoderDecoder with Encoder Attention

In [ ]:
# I ran this several times, and it seemed to produce good results. However, there is a noticeable amount of variance
# Also, maybe its a bit noisy? --> might have to fine-tune a bit more. 
enc_w_attn_args = AttrDict()
args_dict = { 'name': 'enc_w_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': BidirectionalEncoderWithAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithoutAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': False,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 20,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': None,
              'device': 'cpu', ## 'cuda'
              'num_epochs': 200,
              'lr':1e-3,
              'batch_size': 32,
              'lr_decay': 0.97,
              'early_stopping_patience': 20,
              'print': True
}
enc_w_attn_args.update(args_dict)


## EncoderDecoder with Decoder Attention

In [ ]:
# These hyper params seem not that noisy, but variance is noticeable. 
dec_w_attn_args = AttrDict()
args_dict = { 'name': 'dec_w_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': EncoderWithoutAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': False,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 26,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': None, ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': 'Additive',
              'device': 'cpu', ## 'cuda'
              'num_epochs': 200,
              'lr':1e-3,
              'batch_size': 32,
              'lr_decay': 0.80,
              'early_stopping_patience': 20,
              'print': True
}
dec_w_attn_args.update(args_dict)


## EncoderDecoder: Encoder with Attention & Decoder with Attention

In [ ]:
ed_w_attn_args = AttrDict()
args_dict = { 'name': 'ed_w_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': BidirectionalEncoderWithAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': False,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 16,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': 'Scaled-Dot',
              'device': 'cpu', ## 'cuda'
              'num_epochs': 200,
              'lr':1e-3,
              'batch_size': 32,
              'lr_decay': 0.95,
              'early_stopping_patience': 10,
              'print': True
}
ed_w_attn_args.update(args_dict)


## EncoderDecoder with Temporal Attention

In [ ]:
ed_w_temp_attn_args = AttrDict()
args_dict = { 'name': 'ed_w_temp_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': EncoderWithoutAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithoutAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': True,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 16,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': None, ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': None,
              'device': 'cpu', ## 'cuda'
              'num_epochs': 100,
              'lr':1e-3,
              'batch_size': 32,
              'lr_decay': 0.85,
              'early_stopping_patience': 20,
              'print': True
}
ed_w_temp_attn_args.update(args_dict)


## EncoderDecoder with Encoder and Temporal Attention

In [ ]:
enc_and_temp_w_attn_args = AttrDict()
args_dict = { 'name': 'enc_and_temp_w_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': BidirectionalEncoderWithAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithoutAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': True,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 48,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None --- Not Additive or Scaled-Dot works best
              'dec_attn_type': None,
              'device': 'cpu', ## 'cuda'
              'num_epochs': 100,
              'lr':1e-3,
              'batch_size': 10,
              'lr_decay': 0.925,
              'early_stopping_patience': 20,
              'print': True
}
enc_and_temp_w_attn_args.update(args_dict)


## EncoderDecoder with Decoder and Temporal Attention

In [ ]:
dec_and_temp_w_attn_args = AttrDict()
args_dict = { 'name': 'dec_and_temp_w_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': EncoderWithoutAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': True,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 18,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': None, ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': 'Additive', 
              'device': 'cpu', ## 'cuda'
              'num_epochs': 100,
              'lr':1e-3,
              'batch_size': 12,
              'lr_decay': 0.70,
              'early_stopping_patience': 20, 
              'print': True
}
dec_and_temp_w_attn_args.update(args_dict)


## EncoderDecoder with All Attention

In [ ]:
ed_w_all_attn_args = AttrDict()
args_dict = { 'name': 'ed_w_all_attn',
              'model':'ED', ## R
              'rnn_class': None,
              'encoder_class': BidirectionalEncoderWithAttention, ## EncoderWithoutAttention, EncoderWithAttention (only for EncoderDecoder)
              'decoder_class': DecoderWithAttention, ## DecoderWithAttention, DecoderWithoutAttention (only for EncoderDecoder)
              'temporal_attn': True,
              'bidirectional': True,
              'input_size': 15, ## to be replaced
              'output_size': 1, ## to be replaced
              'seq_len': 5, ## to be replaced
              'dropout': 0.,
              'hidden_state_size': 32,
              'encoder_num_hidden_layers': 1,
              'decoder_num_hidden_layers': 1,
              'cell_type': 'LSTM',
              'enc_attn_type': 'Additive', ## Additive, Scaled-Dot, Linear, DA-RNN, None
              'dec_attn_type': 'Additive',
              'device': 'cpu', ## 'cuda'
              'num_epochs': 150,
              'lr':1e-3,
              'batch_size': 32,
              'lr_decay': 0.99, # or maybe 0.9
              'early_stopping_patience': 10,
              'print': True
}
ed_w_all_attn_args.update(args_dict)
# most of the runs seem fine with these parameters, but consistency is sometimes a problem

# Experiments 1 and 2: Model Performance

In [ ]:
"""
BASIC_FEATURES = ['Positive', 'Total Cases', 'Resolved', 'Deaths']
TEST_FEATURES = ['AFT','TTC', 'PPT', 'UI']
HOSPITAL_FEATURES = ['Hospital', 'ICU', 'Ventilator']
LTC_FEATURES = ['TPLTCR', 'TPLTCHCW', 'TLCTRD', 'TLTCHCWD']
"""

OUT_FEATURES = ['Deaths']
ALL_IN_FEATURES = BASIC_FEATURES + TEST_FEATURES + HOSPITAL_FEATURES + LTC_FEATURES
ALL_FEATURES = [BASIC_FEATURES, TEST_FEATURES, HOSPITAL_FEATURES, LTC_FEATURES]
ALL_FEATURE_NAMES = ['Basic', 'Testing', 'Hospital', 'LTC']
MODEL_NAMES = ['rnn_wo_attn', 'rnn_w_input_attn', 'rnn_w_temp_attn', 'rnn_w_attn', 
               'ed_wo_attn', 'dec_w_attn', 'enc_w_attn', 'ed_w_attn', 'ed_w_temp_attn',
               'enc_and_temp_w_attn', 'dec_and_temp_w_attn', 'ed_w_all_attn']
MODEL_ARGS_LST = [rnn_wo_attn_args, rnn_w_input_attn_args, rnn_w_temp_attn_args, 
                  rnn_w_attn_args, ed_wo_attn_args, dec_w_attn_args, enc_w_attn_args, 
                  ed_w_attn_args, ed_w_temp_attn_args, enc_and_temp_w_attn_args, 
                  dec_and_temp_w_attn_args, ed_w_all_attn_args]


def experiments_1_and_2():
    """
    Model performance
    model_args_lst: The AttrDict opts for the models in order
    """
    feature_names_lst = ALL_FEATURE_NAMES
    feature_lst = ALL_FEATURES
    in_features = ALL_IN_FEATURES
    out_features = OUT_FEATURES

    loss_dict = {'model': [], 'mse means': [], 'mse stds': [], 'mae means': [], 'mae stds': []}

    mae_loss_fn = nn.L1Loss()
    for i, model_name in enumerate(MODEL_NAMES):
        args_dict = MODEL_ARGS_LST[i]
        args_dict.print = False

        ## Remove output features from encoder input features if encoder-decoder
        if model_name[:3] != "rnn" and set(in_features) & set(out_features):
            in_features_cpy = in_features.copy()
            for f in out_features:
                if f in in_features_cpy:
                    in_features_cpy.remove(f)
        else:
            in_features_cpy = in_features

        X, X_tilde, y, y_cumul, dates, _ = cluster_data(df, k=5, t=7, separate=True, in_features=in_features_cpy, out_features=out_features, moving_average=True, m=3)
        X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test = train_val_test_split(X, X_tilde, y, dates)

        args_dict.input_size = X_train.shape[2]
        args_dict.output_size = y_train.shape[1]
        args_dict.seq_len = X_train.shape[1]
        
        min_test_losses = np.zeros(10)
        min_test_maes = np.zeros(10)
        for j in range(10):
            train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, args_dict)
            model = torch.load('{}.pt'.format(model_name))
            y_hat = model(X_test.to(args_dict.device), X_tilde_test.to(args_dict.device))
            min_test_losses[j] = compute_loss(y_hat, y_test.to(args_dict.device), nn.MSELoss())
            min_test_maes[j] = mae_loss_fn(y_hat, y_test.to(args_dict.device))
        mse_mean, mse_std = np.mean(min_test_losses), np.std(min_test_losses) 
        mae_mean, mae_std = np.mean(min_test_maes), np.std(min_test_maes)
        loss_dict['mse means'].append(mse_mean)
        loss_dict['mse stds'].append(mse_std)
        loss_dict['mae means'].append(mae_mean)
        loss_dict['mae stds'].append(mae_std)
        loss_dict['model'].append(model_name)
        print("Model: {} | Mean MSE: {} | MSE Std: {} | Mean MAE: {} | MAE Std: {}".format(model_name, mse_mean, mse_std, mae_mean, mae_std))

    return loss_dict

def experiments_1_and_2_plot(loss_dict):
    """Plot a bar graph for model performances """

    fig, ax = plt.subplots()
    y_pos = np.arange(len(MODEL_NAMES))
    ax.barh(y_pos, loss_dict['mse means'], xerr = loss_dict['mse stds'], align = 'center')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(MODEL_NAMES)
    ax.invert_yaxis()
    
    ax.set_xlabel('Mean Test Loss Over 10 Runs')
    plt.savefig('experiment1/plot.pdf')


def print_loss_summary(loss_dict):
    print('Model \t Test MSE (std) \t Test MAE (std)')
    for i, model_name in enumerate(loss_dict['model']):
        print('{} | {} ({}) | {} ({})'.format(model_name, loss_dict['mse means'][i], 
                                              loss_dict['mse stds'][i], loss_dict['mae means'][i], 
                                              loss_dict['mae stds'][i]))

In [ ]:
loss_dict = experiments_1_and_2()

In [ ]:
experiments_1_and_2_plot(loss_dict)

In [ ]:
print_loss_summary(loss_dict)

# Experiment 3 Prelim: Input Features

In [ ]:
OUT_FEATURES = ['Deaths']
ALL_IN_FEATURES = BASIC_FEATURES + TEST_FEATURES + HOSPITAL_FEATURES + LTC_FEATURES
ALL_FEATURES = [BASIC_FEATURES, TEST_FEATURES, HOSPITAL_FEATURES, LTC_FEATURES]
ALL_FEATURE_NAMES = ['Basic', 'Testing', 'Hospital', 'LTC']
MODEL_NAMES = ['rnn_wo_attn', 'rnn_w_temp_attn', 'ed_wo_attn', 'dec_w_attn', 'ed_w_temp_attn', 'dec_and_temp_w_attn']
MODEL_ARGS_LST = [rnn_wo_attn_args, rnn_w_temp_attn_args, ed_wo_attn_args, dec_w_attn_args, ed_w_temp_attn_args, 
                  dec_and_temp_w_attn_args]


def experiment3_prelim():
    """
    Vary the input
    model_args_lst: The AttrDict opts for the models in order
    """
    feature_names_lst = ALL_FEATURE_NAMES
    feature_lst = ALL_FEATURES
    out_features = OUT_FEATURES

    ## Get all feature combinations
    feat_combo_lst = [list(name) for l in range(1, len(feature_names_lst) + 1) for name in combinations(feature_names_lst, l)]

    feat_dict = dict()
    for name in MODEL_NAMES:
        feat_dict[name] = {'mean': [], 'std': []}

    for i, model_name in enumerate(MODEL_NAMES):
        args_dict = MODEL_ARGS_LST[i]
        args_dict.print = False
        args_dict.device = 'cuda'
        for l in range(1, len(feature_lst) + 1):
            for feats in combinations(feature_lst, l):
                in_features = []
                for f in feats:
                    in_features.extend(f)

                ## Remove output features from encoder input features if encoder-decoder
                if model_name[:3] != "rnn" and set(in_features) & set(out_features):
                    in_features_cpy = in_features.copy()
                    for f in out_features:
                        if f in in_features_cpy:
                            in_features_cpy.remove(f)
                else:
                    in_features_cpy = in_features

                X, X_tilde, y, y_cumul, dates, _ = cluster_data(df, k=5, t=7, separate=True, in_features=in_features_cpy, out_features=out_features, moving_average=True, m=3)
                X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test = train_val_test_split(X, X_tilde, y, dates)

                args_dict.input_size = X_train.shape[2]
                args_dict.output_size = y_train.shape[1]
                args_dict.seq_len = X_train.shape[1]
                
                min_test_losses = np.zeros(5)
                for j in range(5):
                    train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, args_dict)
                    model = torch.load('{}.pt'.format(model_name))
                    y_hat = model(X_test.to(args_dict.device), X_tilde_test.to(args_dict.device))
                    min_test_losses[j] = compute_loss(y_hat, y_test.to(args_dict.device), nn.MSELoss())
                loss_mean = np.mean(min_test_losses)
                loss_std = np.std(min_test_losses)
                feat_dict[model_name]['mean'].append(loss_mean)
                feat_dict[model_name]['std'].append(loss_std)
                print("Model: {} | Mean: {} | Std: {}".format(model_name, loss_mean, loss_std))

    return feat_combo_lst, feat_dict

def experiment3_prelim_plot(feat_combo_lst, feat_dict):
    """Plot a bar graph for each model of different combinations of features"""

    for model in MODEL_NAMES:
        fig, ax = plt.subplots()
        y_pos = np.arange(len(feat_combo_lst))
        ax.barh(y_pos, feat_dict[model]['mean'], xerr = feat_dict[model]['std'], align = 'center')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(feat_combo_lst)
        ax.invert_yaxis()
        
        ax.set_xlabel('Mean Test Loss Over 5 Runs')
        ax.set_title('Experiment 3 Prelim: {}'.format(model))
        plt.savefig('experiment3/{}.pdf'.format(model))


In [ ]:
feat_combo_lst, feat_dict = experiment3_prelim()

In [ ]:
experiment3_prelim_plot(feat_combo_lst, feat_dict)

# Experiment 3: Input Attention Visualization

In [ ]:
## Experiment 3
EXP3_MODEL_NAMES = ['rnn_w_input_attn', 'rnn_w_attn', 'enc_w_attn', 'ed_w_attn', 'enc_and_temp_w_attn', 'ed_w_all_attn']
EXP3_MODEL_ARGS_LST = [rnn_w_input_attn_args, rnn_w_attn_args, enc_w_attn_args, ed_w_attn_args, enc_and_temp_w_attn_args, 
                  ed_w_all_attn_args]

## Experiment 4
EXP4_MODEL_NAMES = ['rnn_w_temp_attn', 'rnn_w_attn', 'ed_w_temp_attn',
               'enc_and_temp_w_attn', 'dec_and_temp_w_attn', 'ed_w_all_attn']
EXP4_MODEL_ARGS_LST = [rnn_w_temp_attn_args, rnn_w_attn_args, ed_w_temp_attn_args, enc_and_temp_w_attn_args, 
                  dec_and_temp_w_attn_args, ed_w_all_attn_args]

def experiment_3_or_4_train(experiment3=True):

    if experiment3:
        MODEL_NAMES = EXP3_MODEL_NAMES
        MODEL_ARGS_LST = EXP3_MODEL_ARGS_LST
    else:
        MODEL_NAMES = EXP4_MODEL_NAMES
        MODEL_ARGS_LST = EXP4_MODEL_ARGS_LST

    in_features = ALL_IN_FEATURES
    out_features = OUT_FEATURES

    X_lst, X_tilde_lst = [], []

    for i, model_name in enumerate(MODEL_NAMES):
        args_dict = MODEL_ARGS_LST[i]
        args_dict.print = False
        args_dict.device = 'cuda'

        ## Remove output features from encoder input features if encoder-decoder
        if model_name[:3] != "rnn" and set(in_features) & set(out_features):
            in_features_cpy = in_features.copy()
            for f in out_features:
                if f in in_features_cpy:
                    in_features_cpy.remove(f)
        else:
            in_features_cpy = in_features

        X, X_tilde, y, y_cumul, dates, _ = cluster_data(df, k=5, t=7, separate=True, in_features=in_features_cpy, out_features=out_features, moving_average=True, m=3)
        X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test = train_val_test_split(X, X_tilde, y, dates)

        args_dict.input_size = X_train.shape[2]
        args_dict.output_size = y_train.shape[1]
        args_dict.seq_len = X_train.shape[1]
        
        train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, args_dict)

        if i == 0 or i == len(MODEL_NAMES) - 1: ## Save the inputs for rnn's and ed's
            X_lst.append(X)
            X_tilde_lst.append(X_tilde)
        

    return X_lst, X_tilde_lst, dates

attn_plot_args = AttrDict()
plt_args = {
          'grid_on': True,
          'height': 10,
          'width': 10,
          'plot_num': 1,  # should be a value <= dates.shape[0] and >= 0
          'layer_num': 1, # should be a value <= num layers and >= 0
          'in_features': ALL_IN_FEATURES,
          'plt_encoder': True,
          'plt_temporal': False,
          'plt_forward': True,
          'plt_backward': True,
          'model_type': 'ed',
          'k': 5,
          't': 7,
          'pred_date': None,
          'model_name': None
} 
attn_plot_args.update(plt_args)

def experiment_3_or_4_attn_plots(X_lst, X_tilde_lst, dates, experiment3=True):
    plot_nums = np.array([5, dates.shape[0] // 2, dates.shape[0] - 10])
    if experiment3:
        attn_plot_args.plt_encoder = True
        attn_plot_args.plt_forward = True
        attn_plot_args.plt_backward = True
        attn_plot_args.plt_temporal = False

        MODEL_NAMES = EXP3_MODEL_NAMES
        MODEL_ARGS_LST = EXP3_MODEL_ARGS_LST
    else:
        attn_plot_args.plt_encoder = False
        attn_plot_args.plt_forward = False
        attn_plot_args.plt_backward = False
        attn_plot_args.plt_temporal = True

        MODEL_NAMES = EXP4_MODEL_NAMES
        MODEL_ARGS_LST = EXP4_MODEL_ARGS_LST

    for i, model_name in enumerate(MODEL_NAMES):
        model = torch.load('{}.pt'.format(model_name))

        attn_plot_args.model_name = model_name
        if model_name[:3] == 'rnn':
            j = 0
            model_type = 'rnn'
            attn_plot_args.in_features = ALL_IN_FEATURES
        else:
            j = 1
            model_type = 'ed'
            in_features_cpy = ALL_IN_FEATURES.copy()
            in_features_cpy.remove('Deaths')
            attn_plot_args.in_features = in_features_cpy

        model(X_lst[j].float().to(MODEL_ARGS_LST[i].device), X_tilde_lst[j].float().to(MODEL_ARGS_LST[i].device))
        
        attn_plot_args.plot_num = plot_nums
        attn_plot_args.model_type = model_type
        attn_plot_args.pred_date = dates[plot_nums]

        print(model_name)
        visualize_attention(model, attn_plot_args)


In [ ]:
X_lst, X_tilde_lst, dates = experiment_3_or_4_train()

In [ ]:
experiment_3_or_4_attn_plots(X_lst, X_tilde_lst, dates)

# Experiment 4: Temporal Attention Visualization

In [ ]:
X_lst, X_tilde_lst, dates = experiment_3_or_4_train(experiment3=False)

In [ ]:
experiment_3_or_4_attn_plots(X_lst, X_tilde_lst, dates, experiment3=False)

# Truth vs. Predicted for Best Models

In [ ]:
OUT_FEATURES = ['Deaths']
ALL_IN_FEATURES = BASIC_FEATURES + TEST_FEATURES + HOSPITAL_FEATURES + LTC_FEATURES
MODEL_NAMES = ['rnn_wo_attn', 'rnn_w_input_attn', 'rnn_w_temp_attn', 'rnn_w_attn', 
               'ed_wo_attn', 'dec_w_attn', 'enc_w_attn', 'ed_w_attn', 'ed_w_temp_attn',
               'enc_and_temp_w_attn', 'dec_and_temp_w_attn', 'ed_w_all_attn']
MODEL_ARGS_LST = [rnn_wo_attn_args, rnn_w_input_attn_args, rnn_w_temp_attn_args, 
                  rnn_w_attn_args, ed_wo_attn_args, dec_w_attn_args, enc_w_attn_args, 
                  ed_w_attn_args, ed_w_temp_attn_args, enc_and_temp_w_attn_args, 
                  dec_and_temp_w_attn_args, ed_w_all_attn_args]

def visualize_predictions(model_name):
    ## Train model
    idx = MODEL_NAMES.index(model_name)
    args_dict = MODEL_ARGS_LST[idx]
    assert(args_dict.name == model_name)
    args_dict.print = False
    in_features = ALL_IN_FEATURES
    out_features = OUT_FEATURES

    ## Remove output features from encoder input features if encoder-decoder
    if model_name[:3] != "rnn" and set(in_features) & set(out_features):
        in_features_cpy = in_features.copy()
        for f in out_features:
            if f in in_features_cpy:
                in_features_cpy.remove(f)
    else:
        in_features_cpy = in_features

    X, X_tilde, y, y_cumul, dates, _ = cluster_data(df, k=5, t=7, separate=True, in_features=in_features_cpy, out_features=out_features, moving_average=True, m=3)
    X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, X_test, X_tilde_test, y_test = train_val_test_split(X, X_tilde, y, dates)

    args_dict.input_size = X_train.shape[2]
    args_dict.output_size = y_train.shape[1]
    args_dict.seq_len = X_train.shape[1]
    
    train(X_train, X_tilde_train, y_train, X_val, X_tilde_val, y_val, args_dict)
    model = torch.load('{}.pt'.format(model_name))
    y_hat_train = model(X_train.to(args_dict.device), X_tilde_train.to(args_dict.device))
    y_hat_val = model(X_val.to(args_dict.device), X_tilde_val.to(args_dict.device))
    y_hat_test = model(X_test.to(args_dict.device), X_tilde_test.to(args_dict.device))

    reported_dates = df['Reported Date'].values
    deaths = df['Deaths'].values
    first_date = dates[0]
    first_date_idx = np.where(reported_dates == first_date)[0][0]
    jan_1_idx = np.where(reported_dates == '2021-01-01')[0][0]
    march_1_idx = np.where(reported_dates == '2021-03-01')[0][0]
    april_17_idx = np.where(reported_dates == '2021-04-17')[0][0]

    ## Plot training
    plt.figure()
    y_train = deaths[first_date_idx : jan_1_idx]
    plt.plot(y_train, label="truth")
    plt.plot(y_hat_train.cpu().detach().numpy(), label="predicted")
    plt.title("True and predicted cumulative deaths from May 19 to Dec. 31, 2020")
    plt.xlabel("Number of days since May 19, 2020")
    plt.ylabel("Cumulative Deaths")
    plt.legend()
    plt.show()

    ## Plot validation
    plt.figure()
    y_val = deaths[jan_1_idx : march_1_idx]
    plt.plot(y_val, label="truth")
    plt.plot(y_hat_val.cpu().detach().numpy(), label="predicted")
    plt.title("True and predicted cumulative deaths from Jan. 1 to Feb. 28, 2021")
    plt.xlabel("Number of days since Jan. 1, 2021")
    plt.ylabel("Cumulative Deaths")
    plt.legend()
    plt.show()

    ## Plot test
    plt.figure()
    y_test = deaths[march_1_idx : april_17_idx]
    plt.plot(y_test, label="truth")
    plt.plot(y_hat_test.cpu().detach().numpy(), label="predicted")
    plt.title("True and predicted cumulative deaths from March 1 to April 16, 2021")
    plt.xlabel("Number of days since March 1, 2021")
    plt.ylabel("Cumulative Deaths")
    plt.legend()
    plt.show()



## Encoder-Decoder With Temporal Attention

In [ ]:
visualize_predictions('ed_w_temp_attn')

## RNN Without Attention

In [ ]:
visualize_predictions('rnn_wo_attn')

## Encoder-Decoder With Decoder Attention

In [ ]:
visualize_predictions('dec_w_attn')

## Encoder-Decoder Without Attention

In [ ]:
visualize_predictions('ed_wo_attn')